In [1]:
# Imports the cffi wrapper for the sciencemode library
from sciencemode3 import sciencemode
import time

## Creating same basic variables for the sciencemode library

In [2]:
ack = sciencemode.ffi.new("Smpt_ack*")
device = sciencemode.ffi.new("Smpt_device*")
version_ack = sciencemode.ffi.new("Smpt_get_version_ack*")
ml_init = sciencemode.ffi.new("Smpt_ml_init*")
ml_update = sciencemode.ffi.new("Smpt_ml_update*")
ml_get_current_data = sciencemode.ffi.new("Smpt_ml_get_current_data*")
ml_get_current_data_ack = sciencemode.ffi.new("Smpt_ml_get_current_data_ack*")

## Identifying and Opening the com port

In [3]:
found_stim = False

for i in range(15):
    if found_stim:
        continue
    com_str = "COM%d" % (i + 1)
    com = sciencemode.ffi.new("char[]", com_str.encode("ascii"))
    ret = sciencemode.smpt_check_serial_port(com)
    if (ret):
        print("Port port %s exists" % com_str)
        ret = sciencemode.smpt_open_serial_port(device,com)
        if not ret:
            ret = sciencemode.smpt_close_serial_port(device)
            continue
        ret = sciencemode.smpt_send_get_version_main(device, 0)
        if not ret:
            ret = sciencemode.smpt_close_serial_port(device)
            continue
        cnt = 0
        while ( not sciencemode.smpt_new_packet_received(device) and cnt < 3):
            time.sleep(1)
        if cnt == 3:
            ret = sciencemode.smpt_close_serial_port(device)
            continue
        sciencemode.smpt_last_ack(device, ack);
        if ack.command_number != sciencemode.Smpt_Cmd_Get_Version_Main_Ack:
            ret = sciencemode.smpt_close_serial_port(device)
            continue
        else:
            found_stim = True
            ret = sciencemode.smpt_get_get_version_main_ack(device, version_ack)
if found_stim:
    print("Found and opened a stimulator at %s" % com_str)
    print("Firmware %d.%d.%d" % (version_ack.uc_version.fw_version.major, version_ack.uc_version.fw_version.minor, version_ack.uc_version.fw_version.revision))
    print("SMPT %d.%d.%d" % (version_ack.uc_version.smpt_version.major, version_ack.uc_version.smpt_version.minor, version_ack.uc_version.smpt_version.revision))
    

Port port COM3 exists
Found and opened a stimulator at COM3
Firmware 1.0.2
SMPT 4.0.0


## Initializing Mid Level and stimulating all 4 channels for 10 seconds

In [5]:
# Cleaning up received packages
while ( sciencemode.smpt_new_packet_received(device)):
    sciencemode.smpt_clear_ack(ack)
    sciencemode.smpt_last_ack(device, ack)

ml_init.packet_number = sciencemode.smpt_packet_number_generator_next(device)
ret = sciencemode.smpt_send_ml_init(device, ml_init)
print("smpt_send_ml_init: ", ret)
time.sleep(1)

ml_update.packet_number = sciencemode.smpt_packet_number_generator_next(device)
for i in range(4):
    ml_update.enable_channel[i] = True
    ml_update.channel_config[i].period = 20
    ml_update.channel_config[i].number_of_points = 3
    ml_update.channel_config[i].points[0].time = 100
    ml_update.channel_config[i].points[0].current = 20
    ml_update.channel_config[i].points[1].time = 100
    ml_update.channel_config[i].points[1].current = 0
    ml_update.channel_config[i].points[2].time = 100
    ml_update.channel_config[i].points[2].current = -20

    
ret = sciencemode.smpt_send_ml_update(device, ml_update)
print("smpt_send_ml_update: ", ret)

for i in range(10):
    sciencemode.smpt_clear_ml_get_current_data(ml_get_current_data)
    # this is the correct way in initializing ml_get_current_data
    ml_get_current_data.data_selection[1] = True
    
    ml_get_current_data.packet_number = sciencemode.smpt_packet_number_generator_next(device)
    ret = sciencemode.smpt_send_ml_get_current_data(device, ml_get_current_data)
    time.sleep(1)
    while ( sciencemode.smpt_new_packet_received(device)):
        sciencemode.smpt_clear_ack(ack)
        sciencemode.smpt_last_ack(device, ack)
        if (ack.command_number != sciencemode.Smpt_Cmd_Ml_Get_Current_Data_Ack):
            continue
        sciencemode.smpt_clear_ml_get_current_data_ack(ml_get_current_data_ack)
        ret = sciencemode.smpt_get_ml_get_current_data_ack(device, ml_get_current_data_ack)
        if (not ret):
            print("smpt_get_ml_get_current_data_ack: ", ret)
        error_on_channel = False
        for i in range(4):
            if (ml_get_current_data_ack.stimulation_data.electrode_error[i]):
                error_on_channel = True
                print("error on %d" % i)
        if error_on_channel:
            print("Error on channel")
        else:
            print("All channels ok")
    

packet_number = sciencemode.smpt_packet_number_generator_next(device)
ret = sciencemode.smpt_send_ml_stop(device, packet_number)
print("smpt_send_ml_stop: ", ret)

smpt_send_ml_init:  True
smpt_send_ml_update:  True
All channels ok
All channels ok
error on 3
Error on channel
error on 3
Error on channel
error on 3
Error on channel
error on 3
Error on channel
error on 3
Error on channel
error on 3
Error on channel
error on 3
Error on channel
error on 3
Error on channel
smpt_send_ml_stop:  True


## Closing the opened device port

In [6]:
ret = sciencemode.smpt_close_serial_port(device)
print("smpt_close_serial_port: ", ret)

smpt_close_serial_port:  True
